In [1]:
!pip install stable-baselines3==2.0.0 
!pip install numpy
!pip install tensorflow
%load_ext tensorboard

In [2]:
import tensorflow as tf
import numpy as np
import sys
import matplotlib.pyplot as plt
from IPython.display import clear_output
from stable_baselines3 import SAC
import requests
import datetime
import os
import gymnasium as gym
from gymnasium.spaces import Box
import pandas as pd
from gymnasium import Env

In [3]:
clear_output(wait=True)

try:
  !rm -rf boptestGym
except:
  pass
!git clone -b master https://github.com/ibpsa/project1-boptest-gym.git boptestGym

In [5]:
url = os.getenv("BOPTEST_URL", "http://project1-boptest-web-1:80")
url_api = 'https://api.boptest.net'
test_case = "multizone_office_simple_air"
run_name = "SAC_" + datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
run_dir = os.path.join("local_files", "Saved Models", run_name)
os.makedirs(run_dir, exist_ok=True)
log_path = os.path.join('local_files', 'Logs', run_name)
model_path = os.path.join(run_dir, "SAC_model.zip")

In [6]:
sys.path.insert(0,'boptestGym')
from boptestGymEnv import BoptestGymEnv

## Gettin Setpoint information from CSV

In [8]:
from datetime import datetime
DEFAULT_SETPOINT = 18.0

DAYS = ["MON", "TUE", "WED", "THU", "FRI", "SAT", "SUN"]
DAY_TO_IDX = {d: i for i, d in enumerate(DAYS)}


# Helper functions to roll 15 minutes to next half an hour
def time_to_slot(t):
    """
    Convert time to half-hour slot.
    Rolls 15 minutes forward to next half hour.
    """
    if isinstance(t, str):
        t = datetime.strptime(t, "%H:%M" if len(t) == 5 else "%H:%M:%S")

    minutes = t.hour * 60 + t.minute

    # roll 15 min forward
    if minutes % 30 == 15:
        minutes += 15

    return (minutes // 30) % 48


df = pd.read_csv("bk_setpoint_calendar.csv")

zones = sorted(df["Zone"].unique())
print(zones)
assert len(zones) == 3, "Expected exactly 3 zones"

ZONE_TO_IDX = {z: i for i, z in enumerate(zones)}


calendar = np.full((7, 48, 3), DEFAULT_SETPOINT, dtype=np.float32)


for _, row in df.iterrows():
    day = row["Day"]
    zone = row["Zone"]
    setpoint = row["Setpoint"]

    start_time = row["Start_Time"]
    end_time = row["End_Time"]

    day_idx = DAY_TO_IDX[day]
    zone_idx = ZONE_TO_IDX[zone]

    start_slot = time_to_slot(start_time)
    end_slot = time_to_slot(end_time)

    # Handle overnight wrap
    if end_slot <= start_slot:
        calendar[day_idx, start_slot:, zone_idx] = setpoint
        calendar[day_idx, :end_slot, zone_idx] = setpoint
    else:
        calendar[day_idx, start_slot:end_slot, zone_idx] = setpoint

print("Calendar shape:", calendar.shape)

['Cor', 'Nor', 'Sou']
Calendar shape: (7, 48, 3)


## Custom Reward / Action

In [56]:
class BoptestGymEnvCustomReward(BoptestGymEnv):
    def reset(self, **kwargs):
        obs, info = super().reset(**kwargs)
        self.last_obs = obs
        return obs, info

    def step(self, action):
        obs, reward, terminated, truncated, info = super().step(action)
        self.last_obs = obs
        return obs, reward, terminated, truncated, info
    
    def get_reward(self):
        # Compute BOPTEST core kpis
        kpis = requests.get('{0}/kpi/{1}'.format(self.url, self.testid)).json()['payload']

        ener_rew = 0
        if 0< kpis['ener_tot'] <1:
            ener_rew = 1 - (kpis['ener_tot']*kpis['ener_tot'])
        else:
            ener_rew = - kpis['ener_tot'] * kpis['ener_tot']

        date = super().start_date
        
        # todo: search for best reward function
        reward = - ( (kpis["tdis_tot"] / 1000) + ener_rew) 

        # Record current objective integrand for next evaluation
        self.objective_integrand = reward
        return reward

In [58]:
class AddExpectedSetpointWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        
        old_space: Box = env.observation_space
        assert isinstance(old_space, Box), f"Expected Box, got {type(old_space)}"

        low  = np.concatenate([old_space.low,  np.array([291.0, 291.0, 291.0], dtype=int)]) #18 C
        high = np.concatenate([old_space.high, np.array([297.0, 297.0, 297.0], dtype=int)]) #23 C

        self.observation_space = Box(
            low=low,
            high=high,
            dtype=np.float32
        )

    def observation(self, obs: np.ndarray) -> np.ndarray:
        return np.concatenate([obs, expected_setpoints], axis=-1)

In [60]:
class DeltaTimeWrapper(gym.ActionWrapper):
    def __init__(self, env: Env):
        super().__init__(env)
        assert isinstance(env.action_space, gym.spaces.Box), "RescaleAction expects a Box action space"
        assert np.all(np.isfinite(env.action_space.low)) and np.all(np.isfinite(env.action_space.high)), \
            "Action space must have finite bounds"

        
        self.action_space = gym.spaces.Box(
            low=-2.0, high=2.0, shape=env.action_space.shape, dtype=int
        )
        
    def action(self, action):
        obs = self.env.last_obs
        #add the delta to the observed temp
        return [action[0] / 2 + obs[2], action[1] / 2 + obs[3], action[2] / 2 + obs[4]]
        
    

## The Model Definition

In [ ]:
start_cooling_day = 134 * 24 * 3600  # May 14 
end_cooling_day = 255 * 24 * 3600  # September 12

env = BoptestGymEnvCustomReward(
    url=url,
    testcase=test_case,
    actions=[
        'hvac_oveZonSupCor_TZonHeaSet_u',
        'hvac_oveZonSupNor_TZonHeaSet_u',
        'hvac_oveZonSupSou_TZonHeaSet_u',
    ],
    observations={
        "time": (0,604800),
        "weaSta_reaWeaTDryBul_y": (258, 303), # (-15)/30 C
        "hvac_reaZonCor_TZon_y": (288, 302), # 15/29 C
        "hvac_reaZonNor_TZon_y": (288, 302), # 15/29 C
        "hvac_reaZonSou_TZon_y": (288, 302) # 15/29 C
    },
    random_start_time=True,
    excluding_periods=[(start_cooling_day, end_cooling_day)],
    max_episode_length=7 * 24 * 3600,
    warmup_period=24 * 3600,
    step_period=1800,
    predictive_period=0,
    regressive_period=None,
)

In [ ]:
env = DeltaTimeWrapper(env)

In [ ]:
model = SAC(
        policy='MlpPolicy',
        env=env,
        verbose=1,
        learning_rate=1e-4,
        batch_size=1024,
        tau=0.005,
        gamma=0.99,
        tensorboard_log=log_path,
       seed=42,
    )

In [52]:
from stable_baselines3.common.callbacks import EvalCallback

steps_per_chunk = env.max_episode_length // env.step_period

eval_callback = EvalCallback(
        env,
        best_model_save_path=run_dir,
        log_path="./logs/",
        eval_freq=25 * steps_per_chunk,
        deterministic=True,
        render=False
    )

In [54]:
episodes_no = 25 #can be 25 or 50 as the paper 
total_timesteps = 336 * episodes_no #336 = episode_length / step_period (604800 / 1800 = 336)
model.learn(total_timesteps= total_timesteps) 

Object `info` not found.
Logging to local_files\Logs\SAC_20260114_000118\SAC_3


AttributeError: type object 'super' has no attribute 'start_date'

In [24]:
%tensorboard --logdir ./local_files/Logs --port 6008

In [44]:
env.stop()